#Resumo de texto abstrato com o Amazon Titan

> *Este caderno deve funcionar bem com o kernel **`Data Science 3.0`** no SageMaker Studio*

## Visão geral
Quando trabalhamos com documentos grandes, podemos enfrentar alguns desafios, pois o texto de entrada pode não caber no comprimento do contexto do modelo ou o modelo alucina com documentos grandes, ou por causa de erros de memória etc.

Para solucionar esses problemas, vamos mostrar uma arquitetura que se baseia no conceito de prompts de fragmentação e encadeamento. Esta arquitetura usa o [LangChain](https://python.langchain.com/docs/get_started/introduction.html) que é um framework popular para desenvolvimento de aplicações alimentadas por modelos de linguagem.

### Arquitetura

![](./images/42-text-summarization-2.png)

Nesta arquitetura:

1. Um documento grande (ou um arquivo gigante anexando arquivos pequenos) é carregado
1. O utilitário do LangChain é usado para dividi-lo em vários blocos menores (fragmentação)
1. O primeiro bloco é enviado ao modelo; o modelo retorna o resumo correspondente
1. O LangChain obtém o próximo bloco e o anexa ao resumo retornado, envia o texto combinado como uma nova solicitação ao modelo, o processo se repete até que todos os blocos sejam processados
1. No fim, você tem um resumo final baseado em todo o conteúdo

### Caso de uso
Esta abordagem pode ser usada para resumir transcrições de chamadas, transcrições de reuniões, livros, artigos, postagens do blog e outros conteúdos relevantes.

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from labutils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

## Resumir um texto longo 

### Configurar o LangChain com o Boto3

O LangChain permite acessar o Bedrock após transmitir as informações de sessão do Boto3 ao LangChain. Se você transmitir Nenhum como informação de sessão do Boto3 ao LangChain, o LangChain tentará obter as informações da sessão do seu ambiente.
Para garantir que o cliente correto seja usado, vamos instanciar um graças a um método utilitário.

Você precisará especificar o LLM para a classe do Bedrock do LangChain e pode transmitir os argumentos para inferência. Aqui, você especifica o Amazon Titan Text grande em `model_id` e transmite o parâmetro de inferência do Titan em `textGenerationConfig`.

In [ ]:
from langchain.llms.bedrock import Bedrock
modelId = "amazon.titan-tg1-large"
llm = Bedrock(
    model_id=modelId,
    model_kwargs={
        "maxTokenCount": 4096,
        "stopSequences": [],
        "temperature": 0,
        "topP": 1,
    },
    client=boto3_bedrock,
)

### Carregar um arquivo de texto com muitos tokens

No diretório `letters`, você pode encontrar um arquivo de texto da [Carta do CEO da Amazon às partes interessadas em 2022](https://www.aboutamazon.com/news/company-news/amazon-ceo-andy-jassy-2022-letter-to-shareholders). A célula a seguir carrega o arquivo de texto e conta o número de tokens no arquivo. 

Você verá um aviso indicando que o número de tokens no arquivo de texto excede o número máximo de tokens para este modelo.

In [ ]:
shareholder_letter = "./letters/2022-letter.txt"

with open(shareholder_letter, "r") as file:
    letter = file.read()
    
llm.get_num_tokens(letter)

### Dividir o texto longo em blocos

O texto é muito longo para caber no prompt, então o dividiremos em blocos menores.
`RecursiveCharacterTextSplitter` no LangChain é compatível com a divisão de um texto longo em blocos recursivamente até que o tamanho de cada bloco seja menor do que `chunk_size`. Um texto é separado com `separators=["\n\n", "\n"]` em blocos, o que evita a divisão de cada parágrafo em vários blocos.

Usando 6 mil caracteres por bloco, podemos obter resumos para cada porção separadamente. O número de tokens, ou pedaços de palavras, em um bloco dependerá do texto.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], chunk_size=4000, chunk_overlap=100
)

docs = text_splitter.create_documents([letter])

In [ ]:
num_docs = len(docs)

num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

print(
    f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens"
)

### Resumir e combinar blocos

Presumindo que o número de tokens seja consistente nos outros documentos, não teremos problemas. Vamos usar o LangChain [load_summarize_chain](https://python.langchain.com/en/latest/use_cases/summarization.html) para resumir o texto. `load_summarize_chain` fornece três formas de resumo: `stuff`, `map_reduce` e `refine`. 
- `stuff` coloca todos os blocos em um prompt. Sendo assim, isso atingiria o limite máximo de tokens.
- `map_reduce` sintetiza cada bloco, combina o resumo e depois sintetiza o resumo combinado. Se o resumo combinado for muito grande, ele retornará um erro.
- `refine` resume o primeiro bloco e depois resume o segundo bloco com o primeiro resumo. O mesmo processo se aplica até que todos os blocos sejam resumidos.

`map_reduce` e `refine` invocam o LLM várias vezes e levam um tempo para obter o resumo final. 
Vamos tentar `map_reduce` aqui. 

In [ ]:
# Set verbose=True if you want to see the prompts being used
from langchain.chains.summarize import load_summarize_chain
summary_chain = load_summarize_chain(llm=llm, chain_type="map_reduce", verbose=False)

> ⏰ **Observação:** a depender do número de documentos, da cota de taxa de solicitações do Bedrock e das configurações de novas tentativas definidas, a cadeia abaixo pode levar algum tempo para ser executada.

In [ ]:
output = ""
try:
    
    output = summary_chain.run(docs)

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

In [ ]:
print_ww(output.strip())